## Publish your Collection using the EDC / BC Geoservice

Publish you collections as WMS service. The BC geoservice provides access to publishing your collection as a WMS service using the geoDB Python client. Please refer to the [geoserver](https://docs.geoserver.org/latest/en/user/services/wms/reference.html) documentation for configuring the rest API call to your WMS/geojson service. 

You will have access to this service if you have purchased a large management account. This service is in beta mode. If you find any issues, please report them on our [GitHub Issues page](https://github.com/bcdev/xcube-hub/issues).

## Init the GeoDB client

In [1]:
from xcube_geodb.core.geodb import GeoDBClient
geodb = GeoDBClient()
geodb.whoami


'geodb_965a53df-6c09-4de2-b9ec-1052a2a9534a'

### Create a Database and Copy some Public Data Across

In [2]:
my_database_name = 'my-urban'

In [3]:
geodb.create_database(my_database_name)

<h1>Error: {"hint":null,"details":null,"code":"P0001","message":"Database my-urban exists already."}</h1>

In [84]:
# geodb.drop_collection(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

In [85]:
geodb.copy_collection(collection='SI001L2_LJUBLJANA_UA2018', new_collection='SI001L2_LJUBLJANA_UA2018', database='eea-urban-atlas', new_database=my_database_name)

<h1>Collection copied from eea-urban-atlas/SI001L2_LJUBLJANA_UA2018 to my-urban/SI001L2_LJUBLJANA_UA2018</h1>

In [9]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

GeoDBError: b'{\n  "message": "No collections found in database my-urban",\n  "traceback": "Traceback (most recent call last):\\n  File \\"/home/xcube/xcube_hub/controllers/geoservice.py\\", line 44, in get_collections\\n    collections = geo.get_layers(database_id=database_id, fmt=\'geopandas\')\\n  File \\"/home/xcube/xcube_hub/geoservice.py\\", line 72, in get_layers\\n    return self._provider.get_layers(database_id, fmt)\\n  File \\"/home/xcube/xcube_hub/geoservice.py\\", line 187, in get_layers\\n    raise api.ApiError(404, f\'No collections found in database {database_id}\')\\nxcube_hub.api.ApiError: No collections found in database my-urban\\n"\n}\n'

## Publish your Collection to the BC WMS Service

In [10]:
geodb.publish_gs(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

{'collection_id': 'SI001L2_LJUBLJANA_UA2018',
 'database': 'my-urban',
 'default_style': None,
 'geojson_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=my-urban:my-urban_SI001L2_LJUBLJANA_UA2018&maxFeatures=10&outputFormat=application/json',
 'href': None,
 'name': 'SI001L2_LJUBLJANA_UA2018',
 'preview_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban/wms?service=WMS&version=1.1.0&request=GetMap&layers=my-urban:my-urban_SI001L2_LJUBLJANA_UA2018&bbox=4638735.5,2524979,4704862.5,2594976.25&width=690&height=768&srs=EPSG:3035&styles=&format=application/openlayers',
 'wfs_url': 'https://stage.xcube-geodb.brockmann-consult.de/geoserver/my-urban/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=my-urban%3Amy-urban_SI001L2_LJUBLJANA_UA2018&maxFeatures=10&outputFormat=application%2Fvnd.google-earth.kml%2Bxml'}

You can now click on the preview link whcih will give you an unstyled view of your WMS output.

In [11]:
## List Collections Published on the BC WMS Service

geodb.get_published_gs(database=my_database_name)

,collection_id,database,default_style,geojson_url,href,name,preview_url,wfs_url
0,SI001L2_LJUBLJANA_UA2018,my-urban,None,https://stage.xcube-geodb.brockmann-consult.de...,None,SI001L2_LJUBLJANA_UA2018,https://stage.xcube-geodb.brockmann-consult.de...,https://stage.xcube-geodb.brockmann-consult.de...


### View the Data

Once you have published you collection, you can use the WMS service to visualise your collection. For this using ipyleaflet. In this example we have used a pre-defined style. You can always provide a custom style using the parameter `sld`. We have provided a class that will pass that parameter to the WMS service. Just replace WMSLayer with SldWMSLayer. The `sld`parameter is given as a URL to an SLD file.

In [12]:
url = f"https://stage.xcube-geodb.brockmann-consult.de/geoserver/{my_database_name}/wms?"

In [13]:
from ipyleaflet import Map, WMSLayer, basemaps

In [14]:
from traitlets import Unicode

class SldWMSLayer(WMSLayer):

    sld = Unicode('').tag(sync=True, o=True)


In [15]:
wms = WMSLayer(
    url=url,
    layers='my-urban:my-urban_SI001L2_LJUBLJANA_UA2018',
    format='image/png',
    opacity=0.3,
    attribution='Copernicus',
    styles='ljubilana'
#     sld='https://raw.githubusercontent.com/dzelge/test-gha/main/ljubilana.sld'
)

In [16]:
m = Map(basemap=basemaps.CartoDB.Positron, center=(46.0, 14.5), zoom=13)
m.add_layer(wms)

m

Map(center=[46.0, 14.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In [8]:
geodb.unpublish_gs(collection='SI001L2_LJUBLJANA_UA2018', database=my_database_name)

True